In [3]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import re
import pandas as pd
import urllib
import time
import numpy as np

In [4]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None

In [5]:
def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)

In [6]:
def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [7]:
def getPron(word):
    ods = False #marker for if DOS was used instead of DDS
    print(word)
    if ' ' in word:
        print('had space')
        time.sleep(.300)
        return(np.nan)
    page = getPage(word)
    if page is None:
        print('no page, trying ODS')
        page = simple_get('https://ordnet.dk/ods/ordbog?query=' + urllib.parse.quote(word)) # look in ODS
        ods = True
        if page is None: # if it's still none, return na
            print('no page at all')
            time.sleep(.300)
            return(np.nan)
    page = BeautifulSoup(page)
    page = page.find_all("span", {"class":"lydskrift"})
    if not page:        
        print('no pron in ODD, trying ODS')
        page = simple_get('https://ordnet.dk/ods/ordbog?query=' + urllib.parse.quote(word)) # look in ODS        
        if page is None:
            time.sleep(.300)
            print('no pron data either in ODD or ODS')
            return(np.nan)
        page = BeautifulSoup(page)
        page = page.find_all("span", {'class': 'lydskrift'})
        if not page:
            print('no pron data either in ODD or ODS')
            time.sleep(.300)
            return(np.nan)
        ods = True
    page = page[0]
    rawPron = str(page.get_text())
    time.sleep(.300)
    pron = parseRaw(rawPron)
    if ods:
        pron = str(pron) + "(ods)" # add marker to remember which ones were from the historical dictionary
    print(pron)
    print('------------------------------------------------------')
    return(pron)

In [8]:
def getPage(word):
    urlWord = urllib.parse.quote(word)
    completeURL = 'https://ordnet.dk/ddo/ordbog?query=' + urllib.parse.quote(word)
    print(completeURL)
    return(simple_get(completeURL))

In [9]:
def parseRaw(rawString):
    pattern = re.compile("(?<=\[).+(?=\])")
    pronunciation = pattern.search(rawString)
    if pronunciation is None: # no matches
        return(np.nan)
    return(pronunciation.group(0))

# First stage

In [81]:
danishWords = pd.read_csv('wordsforg2p/manual/danishmanual.csv')

In [82]:
danishProns = [getPron(word) for word in list(danishWords['Orth'])] #still have to debug (timeout?????)
# some are marked as not having but they have it. probably timed out and getting kicked out of the site. run second round of scraping for words marked as nan.

ofte
https://ordnet.dk/ddo/ordbog?query=ofte
ˈʌfdə
------------------------------------------------------
tit
https://ordnet.dk/ddo/ordbog?query=tit
ˈtid
------------------------------------------------------
altid
https://ordnet.dk/ddo/ordbog?query=altid
ˈalˀˌtiðˀ
------------------------------------------------------
færdig
https://ordnet.dk/ddo/ordbog?query=f%C3%A6rdig
ˈfæɐ̯di
------------------------------------------------------
ophøre
https://ordnet.dk/ddo/ordbog?query=oph%C3%B8re
ˈʌbˌhøˀʌ
------------------------------------------------------
ende
https://ordnet.dk/ddo/ordbog?query=ende
ˈεnə
------------------------------------------------------
slutte
https://ordnet.dk/ddo/ordbog?query=slutte
ˈsludə
------------------------------------------------------
af-slutte
https://ordnet.dk/ddo/ordbog?query=af-slutte
ˈɑwˌsludə
------------------------------------------------------
ende
https://ordnet.dk/ddo/ordbog?query=ende
ˈεnə
------------------------------------------------------
slu

ˈliːnənə
------------------------------------------------------
lige
https://ordnet.dk/ddo/ordbog?query=lige
ˈliːə
------------------------------------------------------
linie
https://ordnet.dk/ddo/ordbog?query=linie
ˈlinjə
------------------------------------------------------
hul
https://ordnet.dk/ddo/ordbog?query=hul
ˈhɔl
------------------------------------------------------
klode
https://ordnet.dk/ddo/ordbog?query=klode
ˈkloːðə
------------------------------------------------------
kugle
https://ordnet.dk/ddo/ordbog?query=kugle
ˈkuːlə
------------------------------------------------------
rund
https://ordnet.dk/ddo/ordbog?query=rund
ˈʁɔnˀ
------------------------------------------------------
kreds
https://ordnet.dk/ddo/ordbog?query=kreds
ˈkʁεˀs
------------------------------------------------------
firkant
https://ordnet.dk/ddo/ordbog?query=firkant
ˈfiɐ̯-
------------------------------------------------------
kors
https://ordnet.dk/ddo/ordbog?query=kors
ˈkɒːs
--------------------

ˈsεdə
------------------------------------------------------
stille
https://ordnet.dk/ddo/ordbog?query=stille
ˈsdelə
------------------------------------------------------
lægge
https://ordnet.dk/ddo/ordbog?query=l%C3%A6gge
ˈlεgə
------------------------------------------------------
plads
https://ordnet.dk/ddo/ordbog?query=plads
ˈplas
------------------------------------------------------
sted
https://ordnet.dk/ddo/ordbog?query=sted
ˈsdεð
------------------------------------------------------
oppe
https://ordnet.dk/ddo/ordbog?query=oppe
ˈʌbə
------------------------------------------------------
ovenpå
https://ordnet.dk/ddo/ordbog?query=ovenpa%CC%8A
no page, trying ODS
no page at all
under
https://ordnet.dk/ddo/ordbog?query=under
ˈɔnˀʌ
------------------------------------------------------
ude
https://ordnet.dk/ddo/ordbog?query=ude
ˈuːðə
------------------------------------------------------
inde
https://ordnet.dk/ddo/ordbog?query=inde
ˈenə
-------------------------------------------

no page, trying ODS
no page at all
flåde
https://ordnet.dk/ddo/ordbog?query=fla%CC%8Ade
no page, trying ODS
no page at all
udlægger
https://ordnet.dk/ddo/ordbog?query=udl%C3%A6gger
nan
------------------------------------------------------
kano
https://ordnet.dk/ddo/ordbog?query=kano
ˈkæːno
------------------------------------------------------
båd
https://ordnet.dk/ddo/ordbog?query=ba%CC%8Ad
no page, trying ODS
no page at all
skib
https://ordnet.dk/ddo/ordbog?query=skib
ˈsgiˀb
------------------------------------------------------
åg
https://ordnet.dk/ddo/ordbog?query=a%CC%8Ag
no page, trying ODS
no page at all
aksel
https://ordnet.dk/ddo/ordbog?query=aksel
ˈɑgsəl
------------------------------------------------------
hjul
https://ordnet.dk/ddo/ordbog?query=hjul
ˈjuˀl
------------------------------------------------------
vogn
https://ordnet.dk/ddo/ordbog?query=vogn
ˈvɒwˀn
------------------------------------------------------
bro
https://ordnet.dk/ddo/ordbog?query=bro
ˈbʁoˀ
------

ˈʁɶːʌ
------------------------------------------------------
male
https://ordnet.dk/ddo/ordbog?query=male
ˈmæːlə
------------------------------------------------------
mejsel
https://ordnet.dk/ddo/ordbog?query=mejsel
ˈmɑjˀsəl
------------------------------------------------------
maling
https://ordnet.dk/ddo/ordbog?query=maling
ˈmæːleŋ
------------------------------------------------------
farve
https://ordnet.dk/ddo/ordbog?query=farve
ˈfɑːvə
------------------------------------------------------
statue
https://ordnet.dk/ddo/ordbog?query=statue
ˈsdæˀtuə
------------------------------------------------------
billedhugger
https://ordnet.dk/ddo/ordbog?query=billedhugger
ˈbeləðˌhɔgʌ
------------------------------------------------------
udhugge
https://ordnet.dk/ddo/ordbog?query=udhugge
nan
------------------------------------------------------
vifte
https://ordnet.dk/ddo/ordbog?query=vifte
ˈvefdə
------------------------------------------------------
vifte
https://ordnet.dk/ddo/ordbog?que

...

-ˌfɑwˀnə
------------------------------------------------------
smerte
https://ordnet.dk/ddo/ordbog?query=smerte
ˈsmæɐ̯də
------------------------------------------------------
kummer
https://ordnet.dk/ddo/ordbog?query=kummer
ˈkɔmˀʌ
------------------------------------------------------
sorg
https://ordnet.dk/ddo/ordbog?query=sorg
ˈsɒˀw
------------------------------------------------------
angst
https://ordnet.dk/ddo/ordbog?query=angst
ˈɑŋˀsd
------------------------------------------------------
beklage
https://ordnet.dk/ddo/ordbog?query=beklage
beˈklæˀjə
------------------------------------------------------
angre
https://ordnet.dk/ddo/ordbog?query=angre
ˈɑŋʁʌ
------------------------------------------------------
medlidenhed
https://ordnet.dk/ddo/ordbog?query=medlidenhed
mεðˈliðˀənˌheðˀ
------------------------------------------------------
græde
https://ordnet.dk/ddo/ordbog?query=gr%C3%A6de
ˈgʁaːðə
------------------------------------------------------
tåre
https://ordnet.dk/ddo/

ˈɔnʌˌviˀsə
------------------------------------------------------
elev
https://ordnet.dk/ddo/ordbog?query=elev
eˈleˀw
------------------------------------------------------
lærer
https://ordnet.dk/ddo/ordbog?query=l%C3%A6rer
ˈlεːʌ
------------------------------------------------------
skole
https://ordnet.dk/ddo/ordbog?query=skole
ˈsgoːlə
------------------------------------------------------
huske
https://ordnet.dk/ddo/ordbog?query=huske
ˈhusgə
------------------------------------------------------
mindes
https://ordnet.dk/ddo/ordbog?query=mindes
ˈmenə
------------------------------------------------------
erindre
https://ordnet.dk/ddo/ordbog?query=erindre
eˈʁεnˀdʁʌ
------------------------------------------------------
glemme
https://ordnet.dk/ddo/ordbog?query=glemme
ˈglεmə
------------------------------------------------------
klar
https://ordnet.dk/ddo/ordbog?query=klar
ˈklɑˀ
------------------------------------------------------
tydelig
https://ordnet.dk/ddo/ordbog?query=tydelig
ˈ

ˈpʁɑːlə
------------------------------------------------------
skrive
https://ordnet.dk/ddo/ordbog?query=skrive
ˈsgʁiːvə
------------------------------------------------------
læse
https://ordnet.dk/ddo/ordbog?query=l%C3%A6se
ˈlεːsə
------------------------------------------------------
papir
https://ordnet.dk/ddo/ordbog?query=papir
paˈpiɐ̯ˀ
------------------------------------------------------
pen
https://ordnet.dk/ddo/ordbog?query=pen
ˈpεnˀ
------------------------------------------------------
bog
https://ordnet.dk/ddo/ordbog?query=bog
ˈbɔˀw
------------------------------------------------------
digter
https://ordnet.dk/ddo/ordbog?query=digter
ˈdegdʌ
------------------------------------------------------
fløjte
https://ordnet.dk/ddo/ordbog?query=fl%C3%B8jte
ˈflʌjdə
------------------------------------------------------
tromme
https://ordnet.dk/ddo/ordbog?query=tromme
ˈtʁɔmə
------------------------------------------------------
horn
https://ordnet.dk/ddo/ordbog?query=horn
ˈhoɐ̯ˀn
-

geˈvεˀɐ̯
------------------------------------------------------
bøsse
https://ordnet.dk/ddo/ordbog?query=b%C3%B8sse
ˈbøsə
------------------------------------------------------
kanon
https://ordnet.dk/ddo/ordbog?query=kanon
kaˈnoˀn
------------------------------------------------------
rustning
https://ordnet.dk/ddo/ordbog?query=rustning
no page, trying ODS
no pron in ODD, trying ODS
no pron data either in ODD or ODS
harnisk
https://ordnet.dk/ddo/ordbog?query=harnisk
ˈhɑːnisg
------------------------------------------------------
hjelm
https://ordnet.dk/ddo/ordbog?query=hjelm
ˈjεlˀm
------------------------------------------------------
skjold
https://ordnet.dk/ddo/ordbog?query=skjold
ˈsgjʌlˀ
------------------------------------------------------
fæstning
https://ordnet.dk/ddo/ordbog?query=f%C3%A6stning
ˈfεsdneŋ
------------------------------------------------------
tårn
https://ordnet.dk/ddo/ordbog?query=ta%CC%8Arn
no page, trying ODS
no page at all
sejr
https://ordnet.dk/ddo/ordbog?

ˈhεlvəðə
------------------------------------------------------
ond ånd
had space
afgud
https://ordnet.dk/ddo/ordbog?query=afgud
ˈɑwˌguð
------------------------------------------------------
trolddom
https://ordnet.dk/ddo/ordbog?query=trolddom
ˈtʁʌlˌdʌmˀ
------------------------------------------------------
magi
https://ordnet.dk/ddo/ordbog?query=magi
maˈgiˀ
------------------------------------------------------
heks
https://ordnet.dk/ddo/ordbog?query=heks
ˈhεgs
------------------------------------------------------
troldkvinde
https://ordnet.dk/ddo/ordbog?query=troldkvinde
no page, trying ODS
no pron in ODD, trying ODS
no pron data either in ODD or ODS
fe
https://ordnet.dk/ddo/ordbog?query=fe
no page, trying ODS
no pron in ODD, trying ODS
no pron data either in ODD or ODS
alf
https://ordnet.dk/ddo/ordbog?query=alf
ˈalˀf
------------------------------------------------------
spøgelse
https://ordnet.dk/ddo/ordbog?query=sp%C3%B8gelse
ˈsbøːjəlsə
----------------------------------------

In [84]:
danishWords['Phon'] = danishProns

In [89]:
danishWords.to_csv('danishOrthPhon.csv', encoding='utf-8')

# Second stage

In [12]:
wordsPostScrape = pd.read_csv('danishOrthPhon.csv')

In [13]:
notScraped = wordsPostScrape.loc[(pd.isnull(wordsPostScrape['Phon'])) | (wordsPostScrape['Phon'] == 'nan(ods)'), :]

In [22]:
firstScrape = [getPron(word) for word in list(notScraped['Orth'])] # because of an error in my code, this was actually a second scrape of DDO.

skynde sig
had space
tre gange
had space
to gange
had space
par
https://ordnet.dk/ddo/ordbog?query=par
no pron data
få
https://ordnet.dk/ddo/ordbog?query=fa%CC%8A
no page at all
al
https://ordnet.dk/ddo/ordbog?query=al
no pron data
to
https://ordnet.dk/ddo/ordbog?query=to
no pron data
små
https://ordnet.dk/ddo/ordbog?query=sma%CC%8A
no page at all
måle
https://ordnet.dk/ddo/ordbog?query=ma%CC%8Ale
no page at all
til-lukke
https://ordnet.dk/ddo/ordbog?query=til-lukke
nan
------------------------------------------------------
åbne
https://ordnet.dk/ddo/ordbog?query=a%CC%8Abne
no page at all
lukke op
had space
hobe op
had space
samle op
had space
stå
https://ordnet.dk/ddo/ordbog?query=sta%CC%8A
no page at all
ovenpå
https://ordnet.dk/ddo/ordbog?query=ovenpa%CC%8A
no page at all
ved siden af
had space
dele med
had space
låne
https://ordnet.dk/ddo/ordbog?query=la%CC%8Ane
no page at all
låne
https://ordnet.dk/ddo/ordbog?query=la%CC%8Ane
no page at all
slippe løs
had space
lede efter


nan
------------------------------------------------------
skulderblad
https://ordnet.dk/ddo/ordbog?query=skulderblad
nan
------------------------------------------------------
ørevoks
https://ordnet.dk/ddo/ordbog?query=%C3%B8revoks
nan
------------------------------------------------------
øjenhår
https://ordnet.dk/ddo/ordbog?query=%C3%B8jenha%CC%8Ar
no page at all
øjelåg
https://ordnet.dk/ddo/ordbog?query=%C3%B8jela%CC%8Ag
no page at all
øjenbryn
https://ordnet.dk/ddo/ordbog?query=%C3%B8jenbryn
nan
------------------------------------------------------
hovedskal
https://ordnet.dk/ddo/ordbog?query=hovedskal
nan
------------------------------------------------------
hjerneskal
https://ordnet.dk/ddo/ordbog?query=hjerneskal
nan
------------------------------------------------------
åre
https://ordnet.dk/ddo/ordbog?query=a%CC%8Are
no page at all
skamhår
https://ordnet.dk/ddo/ordbog?query=skamha%CC%8Ar
no page at all
hår
https://ordnet.dk/ddo/ordbog?query=ha%CC%8Ar
no page at all
hår

ʁεˈtʁεːdə
------------------------------------------------------
rov
https://ordnet.dk/ddo/ordbog?query=rov
ˈʁɒw
------------------------------------------------------
fiskekrog
https://ordnet.dk/ddo/ordbog?query=fiskekrog
nan
------------------------------------------------------
fiskenet
https://ordnet.dk/ddo/ordbog?query=fiskenet
nan
------------------------------------------------------
lokkemad
https://ordnet.dk/ddo/ordbog?query=lokkemad
nan
------------------------------------------------------
ramme forbi
had space
anklage
https://ordnet.dk/ddo/ordbog?query=anklage
ˈanˌklæːjə
------------------------------------------------------
straf
https://ordnet.dk/ddo/ordbog?query=straf
ˈsdʁɑf
------------------------------------------------------
mordbrand
https://ordnet.dk/ddo/ordbog?query=mordbrand
ˈmoɐ̯-
------------------------------------------------------
Gud
https://ordnet.dk/ddo/ordbog?query=Gud
ˈguð
------------------------------------------------------
ond ånd
had space
troldkv

In [28]:
import copy
firstScrape = copy.deepcopy(secondScrape)

In [36]:
notScraped['Phon'] = firstScrape

C:\Users\contr\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [40]:
def getPronODS(word):
    print(word)
    if ' ' in word:
        print('had space')
        time.sleep(1)
        return(np.nan)
    page = getPageODS(word)
    if page is None:
        print('no page at all')
        time.sleep(1)
        return(np.nan)
    page = BeautifulSoup(page)
    page = page.find_all("span", {"class":"lydskrift"})
    if not page:        
        time.sleep(1)
        print('no pron data')
        return(np.nan)
    page = page[0]
    rawPron = str(page.get_text())
    time.sleep(1)
    pron = parseRaw(rawPron)
    print(pron)
    print('------------------------------------------------------')
    return(pron)

In [41]:
def getPageODS(word): # query ODS instead of ODD
    urlWord = urllib.parse.quote(word)
    completeURL = 'https://ordnet.dk/ods/ordbog?query=' + urllib.parse.quote(word)
    print(completeURL)
    return(simple_get(completeURL))

In [49]:
forSecondScrape = notScraped['Orth'][pd.isnull(firstScrape)]
secondScrape = [getPronODS(word) for word in forSecondScrape]

skynde sig
had space
tre gange
had space
to gange
had space
par
https://ordnet.dk/ods/ordbog?query=par
nan
------------------------------------------------------
få
https://ordnet.dk/ods/ordbog?query=fa%CC%8A
no page at all
al
https://ordnet.dk/ods/ordbog?query=al
no pron data
to
https://ordnet.dk/ods/ordbog?query=to
no pron data
små
https://ordnet.dk/ods/ordbog?query=sma%CC%8A
no page at all
måle
https://ordnet.dk/ods/ordbog?query=ma%CC%8Ale
no page at all
til-lukke
https://ordnet.dk/ods/ordbog?query=til-lukke
nan
------------------------------------------------------
åbne
https://ordnet.dk/ods/ordbog?query=a%CC%8Abne
no page at all
lukke op
had space
hobe op
had space
samle op
had space
stå
https://ordnet.dk/ods/ordbog?query=sta%CC%8A
no page at all
ovenpå
https://ordnet.dk/ods/ordbog?query=ovenpa%CC%8A
no page at all
ved siden af
had space
dele med
had space
låne
https://ordnet.dk/ods/ordbog?query=la%CC%8Ane
no page at all
låne
https://ordnet.dk/ods/ordbog?query=la%CC%8Ane
n

no pron data
papagøje
https://ordnet.dk/ods/ordbog?query=papag%C3%B8je
no pron data
måge
https://ordnet.dk/ods/ordbog?query=ma%CC%8Age
no page at all
gås
https://ordnet.dk/ods/ordbog?query=ga%CC%8As
no page at all
moderfår
https://ordnet.dk/ods/ordbog?query=moderfa%CC%8Ar
no page at all
får
https://ordnet.dk/ods/ordbog?query=fa%CC%8Ar
no page at all
hun-
https://ordnet.dk/ods/ordbog?query=hun-
no pron data
han-
https://ordnet.dk/ods/ordbog?query=han-
no pron data
enkemand
https://ordnet.dk/ods/ordbog?query=enkemand
no pron data
forældreløs
https://ordnet.dk/ods/ordbog?query=for%C3%A6ldrel%C3%B8s
no pron data
søskendebarn
https://ordnet.dk/ods/ordbog?query=s%C3%B8skendebarn
no pron data
sønnesøn
https://ordnet.dk/ods/ordbog?query=s%C3%B8nnes%C3%B8n
no pron data
dattersøn
https://ordnet.dk/ods/ordbog?query=datters%C3%B8n
no pron data
sønnedatter
https://ordnet.dk/ods/ordbog?query=s%C3%B8nnedatter
no pron data
datterdatter
https://ordnet.dk/ods/ordbog?query=datterdatter
no pron data
g

In [54]:
notScraped.to_csv('backupSecondScrape.csv')

In [59]:
words = pd.read_csv('danishOrthPhon.csv')

In [112]:
for word in notScraped['Orth']:
    print(word)
    index1 = pd.Index(list(words['Orth'])).get_loc(word)
    index2 = pd.Index(list(notScraped['Orth'])).get_loc(word)
    if type(index1) is int:
        words.iloc[index1, 2] = notScraped.iloc[index2, 2]
        print(words.iloc[index1])

skynde sig
Unnamed: 0            14
Orth          skynde sig
Phon                 NaN
Name: 14, dtype: object
tre gange
Unnamed: 0           28
Orth          tre gange
Phon                NaN
Name: 28, dtype: object
to gange
Unnamed: 0          30
Orth          to gange
Phon               NaN
Name: 30, dtype: object
par
Unnamed: 0     31
Orth          par
Phon          NaN
Name: 31, dtype: object
få
al
Unnamed: 0     54
Orth           al
Phon          NaN
Name: 54, dtype: object
to
Unnamed: 0     70
Orth           to
Phon          NaN
Name: 70, dtype: object
små
Unnamed: 0     110
Orth          små
Phon           NaN
Name: 110, dtype: object
måle
Unnamed: 0      113
Orth          måle
Phon            NaN
Name: 113, dtype: object
til-lukke
Unnamed: 0          140
Orth          til-lukke
Phon                NaN
Name: 140, dtype: object
åbne
Unnamed: 0      142
Orth          åbne
Phon            NaN
Name: 142, dtype: object
lukke op
Unnamed: 0         143
Orth          lukke op
Pho

Name: 633, dtype: object
si
Unnamed: 0    638
Orth           si
Phon          NaN
Name: 638, dtype: object
aftensmad
Unnamed: 0          641
Orth          aftensmad
Phon                NaN
Name: 641, dtype: object
måltid
Unnamed: 0        645
Orth          måltid
Phon              NaN
Name: 645, dtype: object
underkop
Unnamed: 0         650
Orth          underkop
Phon               NaN
Name: 650, dtype: object
skål
Unnamed: 0      652
Orth          skål
Phon            NaN
Name: 652, dtype: object
lave mad
Unnamed: 0         665
Orth          lave mad
Phon               NaN
Name: 665, dtype: object
rådden
Unnamed: 0        675
Orth          rådden
Phon              NaN
Name: 675, dtype: object
rå
Unnamed: 0    678
Orth          rå
Phon          NaN
Name: 678, dtype: object
lægemiddel
Unnamed: 0           699
Orth          lægemiddel
Phon                 NaN
Name: 699, dtype: object
år
sår
Unnamed: 0     710
Orth          sår
Phon           NaN
Name: 710, dtype: object
i live

Unnamed: 0           1500
Orth          ramme forbi
Phon                  NaN
Name: 1500, dtype: object
anklage
Unnamed: 0          1513
Orth             anklage
Phon          ˈanˌklæːjə
Name: 1513, dtype: object
straf
Unnamed: 0      1521
Orth           straf
Phon          ˈsdʁɑf
Name: 1521, dtype: object
mordbrand
Unnamed: 0         1527
Orth          mordbrand
Phon             ˈmoɐ̯-
Name: 1527, dtype: object
Gud
Unnamed: 0    1532
Orth           Gud
Phon          ˈguð
Name: 1532, dtype: object
ond ånd
Unnamed: 0        1546
Orth          ond ånd
Phon               NaN
Name: 1546, dtype: object
troldkvinde
Unnamed: 0           1551
Orth          troldkvinde
Phon               ˈtʁʌl-
Name: 1551, dtype: object
fe
Unnamed: 0    1552
Orth            fe
Phon          ˈfeˀ
Name: 1552, dtype: object


In [115]:
words.to_csv('danishOrthPhon.csv', encoding = 'utf-8')